<a href="https://colab.research.google.com/github/anaisidoro/ASOCIO-2025---POwer-Girls/blob/main/Instance9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
!pip install pulp
from pulp import *
import time
# --- INÍCIO DA MEDIÇÃO ---
start_time = time.time()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 97.3 MB/s eta 0:00:00


In [2]:
#Input data
with open('/content/instance9 (1).json') as f:
    instance = json.load(f)

#Sets and parameters
days = ["L", "Ma", "Mi", "J", "V"]
employee = instance['Employees']
desks = instance['Desks']
zones = instance['Zones']
desks_per_zone = instance['Desks_Z']
desks_per_employee = instance['Desks_E']
employee_per_group = instance['Employees_G']
preferable_days = instance['Days_E']
weight = 5
groups = [g for g in employee_per_group]

#Update weight matrix
weights = [[1 for _ in employee] for _ in days]
for e in employee:
    for i in range(len(preferable_days[e])):
        day_index = days.index(preferable_days[e][i])
        weights[day_index][employee.index(e)]=weight

#Identify the employees that can use each desk
employee_per_desk = {}
for desk_id in desks:
    employees_in_desk = []
    for e in employee:
        possible_desks = desks_per_employee[e]
        for m in possible_desks:
            if m == desk_id:
                employees_in_desk.append(e)
    employee_per_desk[desk_id]=employees_in_desk

In [3]:
#Problem Initialization
assignment = LpProblem("StudentChallenge", LpMaximize)

#Decision Variables
x = LpVariable.dicts("x", [(d,e,k) for d in days for e in employee for k in desks if e in employee_per_desk[k]], cat = "Binary")
u = LpVariable.dicts("u", [(d,g) for d in days for g in groups], cat = "Binary")

#Objective Function
assignment += lpSum([weights[days.index(d)][employee.index(e)]*x[d,e, k] for d in days for e in employee for k in desks if e in employee_per_desk[k]])

#Constraint 1 = Each employee must stay at the office at least twice a week
for e in employee:
    assignment += lpSum([x[d,e,k] for d in days for k in desks if e in employee_per_desk[k]]) >= 2

#Constraint 2 = Each desk must be occupied by at most one specific employees at each day
for k in desks:
    for d in days:
        assignment += lpSum([x[d,e,k] for e in employee_per_desk[k]]) <= 1

#Constraint 3 - Each employee must be assigned at most once a day
for d in days:
    for e in employee:
        assignment += lpSum([x[d,e,k] for k in desks if e in employee_per_desk[k]]) <= 1

#Constraint 4 - Each desk must be assigned at most once a day
for k in desks:
    for d in days:
        assignment += lpSum([x[d,e,k] for e in employee if e in employee_per_desk[k]]) <= 1


#Constraint 5 = Employees should not be isolated in a zone
for d in days:
    for z in zones:
        for g in groups:
            for m in employee_per_group[g]:
                assignment += lpSum([x[d,m,k] for k in desks_per_employee[m] if k in desks_per_zone[z]]) <= lpSum([x[d,e,k] for e in employee_per_group[g] if e!=m for k in desks_per_employee[e] if k in desks_per_zone[z]])


#Constraint 6 = Each group should have one meeting per week
for g in groups:
    assignment += lpSum([u[d,g] for d in days]) >= 1

#Constraint 7 = All the members of a group should be present once a week
for d in days:
    for g in groups:
        employees_group = employee_per_group[g]
        assignment += lpSum([x[d,e,k] for e in employees_group for k in desks if e in employee_per_desk[k]]) >= len(employees_group)*u[d,g]

In [4]:
#Solving the model
solve_the_model = assignment.solve(PULP_CBC_CMD(timeLimit=3600, keepFiles=True))
print(f"Status do problema: {LpStatus[solve_the_model]}")

#Print variables
for var in assignment.variables():
    if var.varValue > 0:
        print(f"{var.name}={var.varValue}")

#Print objective function value
print(f"Total Happiness: {value(assignment.objective)}")

Status do problema: Not Solved
u_('J',_'G1')=0.83333333
u_('J',_'G14')=0.71174377
u_('J',_'G15')=0.21099369
u_('J',_'G17')=0.4
u_('J',_'G5')=0.43370229
u_('J',_'G8')=0.33333333
u_('L',_'G10')=0.4
u_('L',_'G11')=0.22556834
u_('L',_'G12')=0.4
u_('L',_'G16')=0.4
u_('L',_'G17')=0.41809261
u_('L',_'G2')=0.33333333
u_('L',_'G3')=0.5
u_('L',_'G7')=0.33333333
u_('L',_'G8')=0.66666667
u_('L',_'G9')=0.38144529
u_('Ma',_'G0')=0.18634335
u_('Ma',_'G13')=0.49226565
u_('Mi',_'G10')=0.4
u_('Mi',_'G12')=0.6
u_('Mi',_'G13')=0.50773435
u_('Mi',_'G16')=0.6
u_('Mi',_'G17')=0.18190739
u_('Mi',_'G3')=0.5
u_('Mi',_'G4')=0.81192494
u_('Mi',_'G5')=0.56629771
u_('Mi',_'G6')=0.8092276
u_('Mi',_'G7')=0.33333333
u_('Mi',_'G9')=0.61855471
u_('V',_'G0')=0.81365665
u_('V',_'G1')=0.16666667
u_('V',_'G10')=0.2
u_('V',_'G11')=0.77443166
u_('V',_'G14')=0.28825623
u_('V',_'G15')=0.78900631
u_('V',_'G2')=0.66666667
u_('V',_'G4')=0.18807506
u_('V',_'G6')=0.1907724
u_('V',_'G7')=0.33333333
x_('J',_'E1',_'D20')=0.25177936
x_(

In [5]:
#Print results organized per day
for d in days:
    print(f"-------------------Day: {d}----------------------")
    for k in desks:
        for e in employee:
            if e in employee_per_desk[k]:
                if x[d, e, k].value() == 1:
                    print(f"Desk: {k}, employee:{e}")
    for g in groups:
        if u[d, g].value() == 1:
            group_employees = employee_per_group[g]
            print(f"Meeting of the day: Group {g} - {group_employees}")

-------------------Day: L----------------------
-------------------Day: Ma----------------------
-------------------Day: Mi----------------------
-------------------Day: J----------------------
-------------------Day: V----------------------


In [6]:
# --- FIM DA MEDIÇÃO ---
end_time = time.time()

# Calcula o tempo total decorrido
time_taken = end_time - start_time

# Imprime o tempo de execução formatado
print(f"\n--- Tempo de Execução do Código ---")
print(f"O processo completo levou {time_taken:.4f} segundos para ser executado.")
print(f"-----------------------------------")


--- Tempo de Execução do Código ---
O processo completo levou 294.3430 segundos para ser executado.
-----------------------------------
